- *Asignatura*: Ingeniería del Conocimiento
- *Curso*: 2020/2021
- *Profesor*: Fco. Javier Martínez de Pisón
- Universidad de La Rioja

# Caso Práctico Final: DSS para la ETAP de Logroño

La *Estación de Tratamiento de Agua Potable de Logroño (TAP Río Iregua)* abastece a una población cercana a los 200.000 habitantes, entre los municipios de Logroño, Alberite, Villamediana y Lardero, además de varios polígonos industriales. Las instalaciones de Lardero, donde se realiza el tratamiento principal, se encuentran atendidas en todo momento, es decir, 24 horas los 365 días del año. En más del cincuenta por ciento del tiempo solamente hay una persona que controla las instalaciones, aunque con los conocimientos suficientes para manejar todos los elementos convencionales que afectan al funcionamiento del proceso.

![](https://www.larioja.org/consorcio-aguas/es/abastecimientos-supramunicipales/bajo-iregua/t-p-bajo-iregua.ficheros/881945-ETAP%20Bajo%20Iregua.jpg)

Existen ocasiones en las que **las condiciones del agua captada en el río cambian de manera muy brusca**. Es el caso de bruscas variaciones en el parámetro de turbidez debidas a tormentas, deshielos o movimientos de tierras cercanos a la captación; o de calidad del agua debido a vertidos incontrolados.

Habitualmente, en estos casos se realizan cambios en las dosis de floculante, cloro, u otros tratamientos que permiten controlar la calidad del agua potable; sin embargo, cuando las condiciones son extremas o la incertidumbre es grande, podría ser útil realizar acciones más drásticas como, por ejemplo:

1.	Detener la entrada de agua a la ETAP durante un número de horas determinado.
2.	Disminuir el caudal de entrada para minimizar el gasto en tratamientos.
3.	Otras posibles acciones de mantenimiento o reparación.

El alcance de este proyecto **comprende el desarrollo de un sistema de apoyo a la toma de decisiones (DSS) en la ETAP Río Iregua que permita predecir el consumo en horas posteriores que ayuden a asegurar que las paradas o disminución de caudal no supongan graves contratiempos en el suministro final de agua**. En definitiva, los objetivos de este proyecto son el desarrollo de modelos predictivos que puedan ser integrados en un sistema inteligente de apoyo a la toma de decisiones en la gestión de aguas de la ETAP de Logroño.

En este proyecto se trabajará con el archivo *'etap_entrenam2008_2009.arff'* para el entrenamiento de los modelos y *'etap_testeo2010.arff'* para testear los modelos finales. Finalmente, se guardará el mejor modelo para poder predecir el consumo probable de cada hora según las condiciones previstas en las horas posteriores.


In [1]:
# Importamos paquetes basicos
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

### Leemos la Base de Datos y Preprocesamos

In [2]:
!wget https://github.com/IC1920/Datasets/raw/master/etap_entrenam2008_2009.arff -O etap_entrenam2008_2009.arff
!wget https://github.com/IC1920/Datasets/raw/master/etap_testeo2010.arff -O etap_testeo2010.arff

--2021-05-02 21:39:04--  https://github.com/IC1920/Datasets/raw/master/etap_entrenam2008_2009.arff
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IC1920/Datasets/master/etap_entrenam2008_2009.arff [following]
--2021-05-02 21:39:04--  https://raw.githubusercontent.com/IC1920/Datasets/master/etap_entrenam2008_2009.arff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 912923 (892K) [text/plain]
Saving to: ‘etap_entrenam2008_2009.arff’

etap_entrenam2008_2 100%[===================>] 891.53K  --.-KB/s    in 0.04s   

2021-05-02 21:39:04 (19.7 MB/s) - ‘etap_entrenam2008_2009.arff’ saved [912923/91

In [3]:
# Leemos los archivos arff
df_train = pd.DataFrame(arff.loadarff("etap_entrenam2008_2009.arff")[0])
df_test = pd.DataFrame(arff.loadarff("etap_testeo2010.arff")[0])

# Eliminamos la b y las comillas de las categóricas
for colname in ['fecha', 'nom_mes', 'festivo', 'nom_diasem']:
    df_train[colname] = df_train[colname].astype(str).str.replace('b','').str.replace("'","")
    df_test[colname] = df_test[colname].astype(str).str.replace('b','').str.replace("'","")

display(df_train.head(3), df_test.head(3))

,fecha,anio,mes,nom_mes,diames,festivo,diasem,nom_diasem,hora,temp,consumo
0,1/1/2008,2008.0,1.0,ene,1.0,TRUE,2.0,mar,1.0,-0.275,771.320
1,1/1/2008,2008.0,1.0,ene,1.0,TRUE,2.0,mar,2.0,-0.750,999.210
2,1/1/2008,2008.0,1.0,ene,1.0,TRUE,2.0,mar,3.0,-1.650,1376.105


,fecha,anio,mes,nom_mes,diames,festivo,diasem,nom_diasem,hora,temp,consumo
0,4/1/2010,2010.0,1.0,ene,4.0,FALSE,1.0,lun,16.0,5.200,1709.175
1,4/1/2010,2010.0,1.0,ene,4.0,FALSE,1.0,lun,17.0,5.275,1507.580
2,4/1/2010,2010.0,1.0,ene,4.0,FALSE,1.0,lun,18.0,5.650,1358.575


Las variables de la base de datos son las siguientes:

1.	Fecha (*fecha*): fecha del registro. Formato Fecha.
2.	Año (*anio*): variable numérica.
3.	Mes (*mes*): variable numérica.
4.	Nombre del mes (*nom_mes*): nombre del mes. Variable categórica.
5.	Día del mes (*diames*): día del mes. Variable numérica.
6.	Día festivo o no (*festivo*): si es un día festivo o no. Variable booleana.
7.	Día de la semana (*diasem*): día de la semana en formato numérico.
8.	Nombre del día de la semana  (*nom_diasem*): nombre del día de la semana. Variable categórica.
9.	Hora (*hora*): franja horaria de suministro (de 0 a 23).
10.	Temperatura ambiental (*temp*): temperatura ambiental en (ºC). Variable numérica.

Y como variable de salida:

11.	Consumo final (*consumo*): consumo de agua en metros cúbicos por hora. Variable numérica a predecir.

Los datos de consumo han sido modificados con el objetivo de preservar la confidencialidad de los mismos.


#  1. Ejercicio Propuesto

A partir del archivo *“etap_entrenam2008_2009.arff”*:

1. Según lo visto en proyectos anteriores, desarrollar modelos de regresión los más precisos posibles del consumo de agua. Para ello, seleccione las variables más importantes que influyen en el modelo. Se recomienda utilizar el año 2008 para entrenamiento y el 2009 para validación simple. Explicar el proceso para obtener el mejor modelo con cada técnica, parámetros ajustados, rango de los mismos, errores obtenidos. Identificar qué técnicas han obtenido mejores modelos, qué variables han sido seleccionadas para cada uno y los parámetros finales del algoritmo. Presentar en una tabla los resultados de los modelos con mejores errores (RMSE y MAE) obtenidos con cada técnica.
2. Extraiga los resultados de predicción con la base de datos *“etap_testeo2010.arff”* de los modelos que han obtenido el menor RMSE de testeo con cada técnica, páselos a EXCEL y presente en unas gráficas los valores reales frente a los valores predichos de los mejores modelos con las tres mejores técnicas. Saque gráficas por meses o semanas, en diferentes épocas del año, para poder ver mejor la diferencia entre los resultados.
3. Presente una tabla con los errores de validación frente a los de testeo para cada técnica. Compare los errores obtenidos con la base de datos de testeo del año 2010 con los errores de validación del 2009. ¿Son similares? ¿Existen diferencias significativas en algún método?
4. Analice los datos extremos (con residuos más elevados) y saque conclusiones sobre la fiabilidad de cada uno de los modelos. Muestre los valores de las entradas que producen esos datos extremos y explique una posible causa que los produzcan.

Podemos prescindir de las variables de nombre de mes o dia, ya que disponemos de los mismos en numérico, y cambiamos el bool de festivo a int.

In [24]:
f = lambda x: 1 if x==True else 0
df_test[['festivo']]=df_test[['festivo']].applymap(f)
df_train[['festivo']]=df_train[['festivo']].applymap(f)

In [22]:

df_test.head()

,fecha,anio,mes,nom_mes,diames,festivo,diasem,nom_diasem,hora,temp,consumo
0,4/1/2010,2010.0,1.0,ene,4.0,0,1.0,lun,16.0,5.200,1709.175
1,4/1/2010,2010.0,1.0,ene,4.0,0,1.0,lun,17.0,5.275,1507.580
2,4/1/2010,2010.0,1.0,ene,4.0,0,1.0,lun,18.0,5.650,1358.575
3,4/1/2010,2010.0,1.0,ene,4.0,0,1.0,lun,19.0,5.875,1297.220
4,4/1/2010,2010.0,1.0,ene,4.0,0,1.0,lun,20.0,6.000,1209.570


In [23]:
df_train.head()

,fecha,anio,mes,nom_mes,diames,festivo,diasem,nom_diasem,hora,temp,consumo
0,1/1/2008,2008.0,1.0,ene,1.0,0,2.0,mar,1.0,-0.275,771.320
1,1/1/2008,2008.0,1.0,ene,1.0,0,2.0,mar,2.0,-0.750,999.210
2,1/1/2008,2008.0,1.0,ene,1.0,0,2.0,mar,3.0,-1.650,1376.105
3,1/1/2008,2008.0,1.0,ene,1.0,0,2.0,mar,4.0,-2.100,1726.705
4,1/1/2008,2008.0,1.0,ene,1.0,0,2.0,mar,5.0,-2.550,1919.535


# 2. Uso de los Modelos para la Toma de Decisiones ante un Vertido de Gasóleo en el Iregua

El 1 de Julio de 2010, nos avisan que un camión cisterna ha sufrido un accidente a las 14:00 y ha derramado gasóleo en el río Iregua a la altura de Panzares. 

![](https://e00-elmundo.uecdn.es/elmundo/imagenes/2010/06/16/1276675934_0.jpg)

Se dispone de tres horas, aproximadamente, para tomar una decisión antes de que el agua contaminada llegue a la zona de captación de aguas de la ETAP. Finalmente, **se decide cerrar la entrada de agua a las 16:00 y mantenerla cerrada hasta las 8:00 de la mañana del día siguiente, o sea de 16:00 a 7:59 (16 horas cerrada), que es cuando entra el nuevo turno de trabajo, de modo que se abrirá la esclusa a las 8.00 de la mañana**. De este modo, se espera que exista suficiente tiempo para retirar el camión accidentado y que la parte más voluminosa del vertido sobrepase la entrada a la ETAP. 

Los gestores de la ETAP y responsables de Medio Ambiente y Aguas, están preocupados por saber si el agua almacenada en la ETAP va a ser suficiente para suministrar a Logroño y pueblos adyacentes en esas horas en que la entrada está cerrada. Para ello, **se nos solicita que estimemos el consumo total de agua durante esas horas considerando que la temperatura ambiental prevista es la que aparece en la base de datos “etap_testeo2010.arff”**.

Presente una tabla y unos gráficos con el consumo por horas y el consumo acumulado.

Suponiendo que disponemos de 35000 metros cúbicos, ¿Cuál es la diferencia entre el consumo y el agua almacenada que disponemos? ¿Es posible cerrar esas horas sin que la ciudad quede desabastecida? 

Justifique los resultados frente a los responsables de Medio Ambiente de la Ciudad

# ENTREGA

Recuerda guardar tus cambios en tu repositorio utilizando la opción "Save a copy in GitHub..." del menú File.